In [33]:
# df = spark.read.format('json').load("hdfs://orion11:33001/reddit/sampled_reddit/part-00000-e97c075c-b0b4-49da-b85b-f77c115b75b6-c000.json")
df = spark.read.format('json').load("hdfs://orion11:33001/reddit/sampled_reddit/*")
print(df.count())

309199315


In [34]:
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime, desc
from pyspark.sql.types import TimestampType, DateType

botExpr = "[bB][oO][tT]"

df2 = (df
        .filter(~(df.body.like("[deleted]") | df.author.rlike(botExpr))))

In [35]:
df2.show()

+--------+----------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+----------+--------------+-------+------------+-----+-----+------------+--------+-----------------+------------+---+------------+
|archived|          author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id| link_id|mod_reports|      name| parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|        subreddit|subreddit_id|ups|user_reports|
+--------+----------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+----------+--------------+-------+------------+-----+-----+------

In [36]:
df3 = df2.select("subreddit", "author", "score")

In [37]:
df3.show()

+-----------------+----------------+-----+
|        subreddit|          author|score|
+-----------------+----------------+-----+
|        AskReddit|      halftone84|    1|
|        AskReddit|       keiyakins|    1|
|         politics|  chuck_finley17|    3|
|        AskReddit|          iceage|    1|
|             IAmA| phenomenonymous|    1|
|       California|tumbleweed1993sf|    1|
|             pics|         zero_iq|    2|
|       reddit.com| gabrielbenjamin|    1|
|        AskReddit|         Ishkabo|    5|
|        AskReddit|        ihrtgngr|    2|
|           trance|       Gareth321|    1|
|           google|         BigDisk|    0|
|             IAmA|      robdidlyob|    2|
|         sandiego|       danxmason|    1|
|       googleplus|          LeBrad|    1|
|       technology|        DoubleKK|    1|
|        worldnews|       [deleted]|    2|
|DrawingFromWithin|        Ebeforei|    3|
|           gaming|   ultimatechase|    1|
|       reddit.com|           pilaf|    2|
+----------

In [38]:
from pyspark.sql.functions import countDistinct, sum


df4 = df3.groupBy("author").agg(countDistinct("subreddit").alias("subreddit_count"), sum("score").alias("score_sum")).orderBy(["subreddit_count", "score_sum"], ascending=False)

In [39]:
df4.show()

+--------------------+---------------+---------+
|              author|subreddit_count|score_sum|
+--------------------+---------------+---------+
|           [deleted]|          85450| 65310970|
|      TotesMessenger|           6703|    95716|
|       AutoModerator|           5976|  1516282|
|         TweetPoster|           4880|   116744|
|OriginalPostSearcher|           4213|    16560|
|    ConvertsToMetric|           3125|    18493|
|            autotldr|           2316|    62272|
|         XPostLinker|           1913|     5309|
|    xkcd_transcriber|           1808|    38576|
|    Mentioned_Videos|           1724|    22393|
|    imgurtranscriber|           1667|    45881|
|      User_Simulator|           1451|    36714|
|       ImageRehoster|           1275|     4984|
|        untouchedURL|           1184|     7207|
|        tweet_poster|            877|    12878|
|           changetip|            867|    11058|
|           pixels625|            862|     1971|
| TwitterToStreamabl

In [40]:
df5 = df4.filter(~(df4.author.like("[deleted]")))

In [41]:
df5.show()

+--------------------+---------------+---------+
|              author|subreddit_count|score_sum|
+--------------------+---------------+---------+
|      TotesMessenger|           6703|    95716|
|       AutoModerator|           5976|  1516282|
|         TweetPoster|           4880|   116744|
|OriginalPostSearcher|           4213|    16560|
|    ConvertsToMetric|           3125|    18493|
|            autotldr|           2316|    62272|
|         XPostLinker|           1913|     5309|
|    xkcd_transcriber|           1808|    38576|
|    Mentioned_Videos|           1724|    22393|
|    imgurtranscriber|           1667|    45881|
|      User_Simulator|           1451|    36714|
|       ImageRehoster|           1275|     4984|
|        untouchedURL|           1184|     7207|
|        tweet_poster|            877|    12878|
|           changetip|            867|    11058|
|           pixels625|            862|     1971|
| TwitterToStreamable|            840|    13021|
|             antdud

In [45]:
df6 = df5.limit(100)

In [ ]:
df7 = df6.toPandas()

In [ ]:
df7.plot(x='subreddit_count', y='score_sum')